# Развитие игровой индустрии в начале XXI века

- Автор: Косачев В. А.
- Дата: 2025-08-24

### Цели и задачи проекта

<font color='#777778'>Цель: изучить развитие игровой индустрии с 2000 по 2013 год.
    
Задачи: 
- Изучить и предобработать данные
- Отобрать данные за 2000 - 2013 гг.
- Разделить все игры по оценкам пользователей и критиков, выделить топ-7 платформ по количеству выпущенных игр</font>

### Описание данных

<font color='#777778'>Данные содержат информацию о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр:
- Name — название игры.
- Platform — название платформы.
- Year of Release — год выпуска игры.
- Genre — жанр игры.
- NA sales — продажи в Северной Америке (в миллионах проданных копий).
- EU sales — продажи в Европе (в миллионах проданных копий).
- JP sales — продажи в Японии (в миллионах проданных копий).
- Other sales — продажи в других странах (в миллионах проданных копий).
- Critic Score — оценка критиков (от 0 до 100).
- User Score — оценка пользователей (от 0 до 10).
- Rating — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.</font>

### Содержимое проекта

<font color='#777778'>
    
1. [Загрузка данных и знакомство с ними](#Загрузка-данных-и-знакомство-с-ними)
2. [Проверка ошибок в данных и их предобработка](#Проверка-ошибок-в-данных-и-их-предобработка)
    1. [Названия, или метки, столбцов датафрейма](#Названия,-или-метки,-столбцов-датафрейма)
    2. [Типы данных](#Типы-данных)
    3. [Наличие пропусков в данных](#Наличие-пропусков-в-данных)
    4. [Явные и неявные дубликаты в данных](#Явные-и-неявные-дубликаты-в-данных)
3. [Фильтрация данных](#Фильтрация-данных)
4. [Категоризация данных](#Категоризация-данных)
5. [Итоговый вывод](#Итоговый-вывод)</font>

---

## Загрузка данных и знакомство с ними

- Загрузим необходимые библиотеки Python и данные датасета.


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')

- Познакомимся с данными: выведем первые строки и результат метода `info()`.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


In [4]:
df.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


- Данные содержатся в 16 956 строчках. Пропуски отсутствуют только в столбцах с информацией о платформе и продажах игр ('Names', 'NA sales', 'EU sales', 'JP sales'). В некоторых столбцах, в которых содержится количественная информация, тип данных -- 'object'. В столбцах с тип данных 'float64' можно уменьшить использование памяти.


---

##  Проверка ошибок в данных и их предобработка


### Названия, или метки, столбцов датафрейма

- Выведем на экран названия всех столбцов датафрейма и проверим их стиль написания.
- Приведем все столбцы к стилю snake case.

In [5]:
df.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

In [6]:
df.columns = df.columns.str.replace(' ', '_').str.lower()

In [7]:
df.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

### Типы данных

- В столбце с годом выпуска заменим пропуски на 0

In [8]:
df['year_of_release'] = df['year_of_release'].fillna(0).astype('int64')

- Узнаем, почему поля с продажами в Европе и Японии и оценками пользователей имеют тип 'object'

In [9]:
df['eu_sales'].unique()

array(['28.96', '3.58', '12.76', '10.93', '8.89', '2.26', '9.14', '9.18',
       '6.94', '0.63', '10.95', '7.47', '6.18', '8.03', '4.89', '8.49',
       '9.09', '0.4', '3.75', '9.2', '4.46', '2.71', '3.44', '5.14',
       '5.49', '3.9', '5.35', '3.17', '5.09', '4.24', '5.04', '5.86',
       '3.68', '4.19', '5.73', '3.59', '4.51', '2.55', '4.02', '4.37',
       '6.31', '3.45', '2.81', '2.85', '3.49', '0.01', '3.35', '2.04',
       '3.07', '3.87', '3.0', '4.82', '3.64', '2.15', '3.69', '2.65',
       '2.56', '3.11', '3.14', '1.94', '1.95', '2.47', '2.28', '3.42',
       '3.63', '2.36', '1.71', '1.85', '2.79', '1.24', '6.12', '1.53',
       '3.47', '2.24', '5.01', '2.01', '1.72', '2.07', '6.42', '3.86',
       '0.45', '3.48', '1.89', '5.75', '2.17', '1.37', '2.35', '1.18',
       '2.11', '1.88', '2.83', '2.99', '2.89', '3.27', '2.22', '2.14',
       '1.45', '1.75', '1.04', '1.77', '3.02', '2.75', '2.16', '1.9',
       '2.59', '2.2', '4.3', '0.93', '2.53', '2.52', '1.79', '1.3', '2.6',
   

In [10]:
df['jp_sales'].unique()

array(['3.77', '6.81', '3.79', '3.28', '10.22', '4.22', '6.5', '2.93',
       '4.7', '0.28', '1.93', '4.13', '7.2', '3.6', '0.24', '2.53',
       '0.98', '0.41', '3.54', '4.16', '6.04', '4.18', '3.84', '0.06',
       '0.47', '5.38', '5.32', '5.65', '1.87', '0.13', '3.12', '0.36',
       '0.11', '4.35', '0.65', '0.07', '0.08', '0.49', '0.3', '2.66',
       '2.69', '0.48', '0.38', '5.33', '1.91', '3.96', '3.1', '1.1',
       '1.2', '0.14', '2.54', '2.14', '0.81', '2.12', '0.44', '3.15',
       '1.25', '0.04', '0.0', '2.47', '2.23', '1.69', '0.01', '3.0',
       '0.02', '4.39', '1.98', '0.1', '3.81', '0.05', '2.49', '1.58',
       '3.14', '2.73', '0.66', '0.22', '3.63', '1.45', '1.31', '2.43',
       '0.7', '0.35', '1.4', '0.6', '2.26', '1.42', '1.28', '1.39',
       '0.87', '0.17', '0.94', '0.19', '0.21', '1.6', '0.16', '1.03',
       '0.25', '2.06', '1.49', '1.29', '0.09', '2.87', '0.03', '0.78',
       '0.83', '2.33', '2.02', '1.36', '1.81', '1.97', '0.91', '0.99',
       '0.95', '2.0'

In [11]:
df['user_score'].unique()

array(['8', nan, '8.3', '8.5', '6.6', '8.4', '8.6', '7.7', '6.3', '7.4',
       '8.2', '9', '7.9', '8.1', '8.7', '7.1', '3.4', '5.3', '4.8', '3.2',
       '8.9', '6.4', '7.8', '7.5', '2.6', '7.2', '9.2', '7', '7.3', '4.3',
       '7.6', '5.7', '5', '9.1', '6.5', 'tbd', '8.8', '6.9', '9.4', '6.8',
       '6.1', '6.7', '5.4', '4', '4.9', '4.5', '9.3', '6.2', '4.2', '6',
       '3.7', '4.1', '5.8', '5.6', '5.5', '4.4', '4.6', '5.9', '3.9',
       '3.1', '2.9', '5.2', '3.3', '4.7', '5.1', '3.5', '2.5', '1.9', '3',
       '2.7', '2.2', '2', '9.5', '2.1', '3.6', '2.8', '1.8', '3.8', '0',
       '1.6', '9.6', '2.4', '1.7', '1.1', '0.3', '1.5', '0.7', '1.2',
       '2.3', '0.5', '1.3', '0.2', '0.6', '1.4', '0.9', '1', '9.7'],
      dtype=object)

В данных полях присутствуют значения 'unknown', заменим их на пропуск. 

Значение 'tbd' скорее всего означает о том, что игра еще не вышла на момент выгрузки данных, и рейтинг еще не был составлен.

In [12]:
df[df['user_score'] == 'tbd'].head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
119,Zumba Fitness,Wii,2010,Sports,3.45,2.59,0.0,0.66,NaN,tbd,E
302,Namco Museum: 50th Anniversary,PS2,2005,Misc,2.08,1.35,0.0,0.54,61.0,tbd,E10+
522,Zumba Fitness 2,Wii,2011,Sports,1.51,1.03,0.0,0.27,NaN,tbd,T
647,uDraw Studio,Wii,2010,Misc,1.65,0.57,0.0,0.20,71.0,tbd,E
659,Frogger's Adventures: Temple of the Frog,GBA,0,Adventure,2.15,0.18,0.0,0.07,73.0,tbd,E
...,...,...,...,...,...,...,...,...,...,...,...
16935,Planet Monsters,GBA,2001,Action,0.01,0.0,0.0,0.00,67.0,tbd,E
16937,Bust-A-Move 3000,GC,2003,Puzzle,0.01,0.0,0.0,0.00,53.0,tbd,E
16938,Mega Brain Boost,DS,2008,Puzzle,0.01,0.0,0.0,0.00,48.0,tbd,E
16945,Plushees,DS,2008,Simulation,0.01,0.0,0.0,0.00,NaN,tbd,E


Оказалось, что 'tbd' не зависит от года выпуска, можно считать это значение за 'nan'

- Преобразуем данные столбцы в нужный формат данных

In [13]:
df['eu_sales'] = pd.to_numeric(df['eu_sales'], errors = 'coerce')
df['eu_sales'] = df['eu_sales'].round(2)
df['jp_sales'] = pd.to_numeric(df['jp_sales'], errors = 'coerce')
df['jp_sales'] = df['jp_sales'].round(2)
df['user_score'] = pd.to_numeric(df['user_score'], errors = 'coerce')

- В конце уменьшим потребление памяти в столбце с годом выпуска

In [14]:
df['year_of_release'] = pd.to_numeric(df['year_of_release'], downcast = 'integer')

### Наличие пропусков в данных

- Посчитаем количество пропусков в каждом столбце в абсолютных и относительных значениях.


In [15]:
df.isna().sum()

name                  2
platform              0
year_of_release       0
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

In [16]:
df.isna().mean()

name               0.000118
platform           0.000000
year_of_release    0.000000
genre              0.000118
na_sales           0.000000
eu_sales           0.000354
jp_sales           0.000236
other_sales        0.000000
critic_score       0.513918
user_score         0.546591
rating             0.405225
dtype: float64

- В столбце 'name' и 'genre' не должно быть пропусков. Так как их немного, можно исключить их из анализа. 
- В столбце 'year_of_release' пропуски уже были изменены на индикатор 0.
- В столбцах 'eu_sales' и 'jp-sales' пропуски возможны, так как некоторые игры могли не продаваться официально в некоторых частях света. Заменим их на среднее значение в зависимости от платформы и года выхода игры.
- В столбцах 'critic_score' и 'user_score' значительно много пропусков (более половины от всего числа строчек), возможно на некоторые игры просто не было достаточного количества рецензий или это ошибка сбора данных. Заменять их на среднее было бы несправедливо, так что оставим их без изменений.
- В столбце 'rating' пропуски скорее всего связаны с годом выхода игры -- игра могла выйти раньше, чем приняли регуляцию. Так как формат строчный, можно оставить без изменений.

In [17]:
df = df.dropna(subset = ['name'])

In [18]:
def mean_group_eu_sales(row):
    if pd.isna(row['eu_sales']):
        group = df[(df['platform'] == row['platform']) & 
                   (df['year_of_release'] == row['year_of_release'])]
        return group['eu_sales'].mean()
    else:
        return row['eu_sales']

    
df.apply(mean_group_eu_sales, axis=1)

0        28.96
1         3.58
2        12.76
3        10.93
4         8.89
         ...  
16951     0.00
16952     0.01
16953     0.00
16954     0.00
16955     0.00
Length: 16954, dtype: float64

In [19]:
def mean_group_jp_sales(row):
    if pd.isna(row['jp_sales']):
        group = df[(df['platform'] == row['platform']) & 
                   (df['year_of_release'] == row['year_of_release'])]
        return group['jp_sales'].mean()
    else:
        return row['jp_sales']

    
df.apply(mean_group_jp_sales, axis=1)

0         3.77
1         6.81
2         3.79
3         3.28
4        10.22
         ...  
16951     0.01
16952     0.00
16953     0.01
16954     0.00
16955     0.01
Length: 16954, dtype: float64

Пропуски не были заполнены, значит схожих игр по платформе и году выпуска не нашлось, оставим 0 вместо пропусков

In [20]:
df['eu_sales'] = df['eu_sales'].fillna(0)
df['jp_sales'] = df['jp_sales'].fillna(0)

### Явные и неявные дубликаты в данных

- Изучим уникальные значения в категориальных данных. Проверим, встречаются ли среди данных неявные дубликаты, связанные с опечатками или разным способом написания.
- Проведем нормализацию данных с текстовыми значениями.

In [21]:
df['platform'].unique()

array(['Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA',
       'PS4', '3DS', 'N64', 'PS', 'XB', 'PC', '2600', 'PSP', 'XOne',
       'WiiU', 'GC', 'GEN', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

- Приведем все плафтормы к нижнему регистру

In [22]:
df['platform'] = df['platform'].str.lower()

In [23]:
df['genre'].unique()

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy', 'MISC', 'ROLE-PLAYING', 'RACING', 'ACTION', 'SHOOTER',
       'FIGHTING', 'SPORTS', 'PLATFORM', 'ADVENTURE', 'SIMULATION',
       'PUZZLE', 'STRATEGY'], dtype=object)

- Приведем все жанры к нижнему регистру

In [24]:
df['genre'] = df['genre'].str.lower()

In [25]:
df['year_of_release'].unique()

array([2006, 1985, 2008, 2009, 1996, 1989, 1984, 2005, 1999, 2007, 2010,
       2013, 2004, 1990, 1988, 2002, 2001, 2011, 1998, 2015, 2012, 2014,
       1992, 1997, 1993, 1994, 1982, 2016, 2003, 1986, 2000,    0, 1995,
       1991, 1981, 1987, 1980, 1983], dtype=int16)

В столбце с годами дубликатов нет.

In [26]:
df['rating'].unique()

array(['E', nan, 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'], dtype=object)

- Рейтинги E и K-A одинаковы по смыслу, приравняем к E.

In [27]:
df['rating'] = df['rating'].str.replace('K-A', 'E')

- Проверим наличие явных дубликатов в данных.

In [28]:
df.duplicated().sum() 

241

- Был найден 241 явный дубликат после нормализации данных в столбцах, удалим их.

In [29]:
df = df.drop_duplicates()

In [30]:
print(f"Количество удаленных строк в датафрейме после обработки: {16956 - df.shape[0]}")
print(f'В относительном значении: {1 - df.shape[0]/16956}')

Количество удаленных строк в датафрейме после обработки: 243
В относительном значении: 0.014331210191082855


***После проведения предобработки данных были удалены 2 строки данных с пропусками и 241 явный дубликат.***

---

## Фильтрация данных

Отберем данные по периоду с 2000 по 2013 гг включительно.

In [31]:
df_actual = df[(df['year_of_release'] >= 2000) & (df['year_of_release'] <= 2013)]
df_actual = df_actual.sort_values(by = 'year_of_release')
df_actual.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
2743,Mega Man X5,ps,2000,platform,0.30,0.21,0.20,0.05,76.0,8.9,E
6558,Star Trek: Invasion,ps,2000,simulation,0.15,0.10,0.00,0.02,76.0,NaN,E
11063,ESPN NBA 2Night,ps2,2000,sports,0.05,0.04,0.00,0.01,62.0,NaN,E
6586,Star Wars Episode I: Battle for Naboo,n64,2000,simulation,0.21,0.05,0.00,0.00,NaN,NaN,NaN
1726,Mario Tennis,gb,2000,sports,0.50,0.18,0.44,0.06,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
724,Assassin's Creed IV: Black Flag,xone,2013,action,1.48,0.55,0.00,0.21,NaN,7.4,M
16175,Storm Lover 2nd,psp,2013,misc,0.00,0.00,0.02,0.00,NaN,NaN,NaN
727,Madden NFL 25,x360,2013,sports,1.98,0.06,0.00,0.19,80.0,5.6,E
16162,White Album 2: Shiawase no Mukougawa,psv,2013,adventure,0.00,0.00,0.02,0.00,NaN,NaN,NaN


---

## Категоризация данных
    
Проведем категоризацию данных:
- Разделим все игры по оценкам пользователей и выделим такие категории: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала).

In [32]:
df_actual['user_score_category'] = pd.cut(df_actual['user_score'], bins = [0, 3, 8, 10.01], right = False, labels = ['низкая оценка', 'средняя оценка', 'высокая оценка'])
df_actual.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_score_category
2743,Mega Man X5,ps,2000,platform,0.30,0.21,0.20,0.05,76.0,8.9,E,высокая оценка
6558,Star Trek: Invasion,ps,2000,simulation,0.15,0.10,0.00,0.02,76.0,NaN,E,NaN
11063,ESPN NBA 2Night,ps2,2000,sports,0.05,0.04,0.00,0.01,62.0,NaN,E,NaN
6586,Star Wars Episode I: Battle for Naboo,n64,2000,simulation,0.21,0.05,0.00,0.00,NaN,NaN,NaN,NaN
1726,Mario Tennis,gb,2000,sports,0.50,0.18,0.44,0.06,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
724,Assassin's Creed IV: Black Flag,xone,2013,action,1.48,0.55,0.00,0.21,NaN,7.4,M,средняя оценка
16175,Storm Lover 2nd,psp,2013,misc,0.00,0.00,0.02,0.00,NaN,NaN,NaN,NaN
727,Madden NFL 25,x360,2013,sports,1.98,0.06,0.00,0.19,80.0,5.6,E,средняя оценка
16162,White Album 2: Shiawase no Mukougawa,psv,2013,adventure,0.00,0.00,0.02,0.00,NaN,NaN,NaN,NaN


- Разделим все игры по оценкам критиков и выделим такие категории: высокая оценка (от 80 до 100 включительно), средняя оценка (от 30 до 80, не включая правую границу интервала) и низкая оценка (от 0 до 30, не включая правую границу интервала).

In [33]:
df_actual['critic_score_category'] = pd.cut(df_actual['critic_score'], bins = [0, 30, 80, 100.01], right = False, labels = ['низкая оценка', 'средняя оценка', 'высокая оценка'])
df_actual.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_score_category,critic_score_category
2743,Mega Man X5,ps,2000,platform,0.30,0.21,0.20,0.05,76.0,8.9,E,высокая оценка,средняя оценка
6558,Star Trek: Invasion,ps,2000,simulation,0.15,0.10,0.00,0.02,76.0,NaN,E,NaN,средняя оценка
11063,ESPN NBA 2Night,ps2,2000,sports,0.05,0.04,0.00,0.01,62.0,NaN,E,NaN,средняя оценка
6586,Star Wars Episode I: Battle for Naboo,n64,2000,simulation,0.21,0.05,0.00,0.00,NaN,NaN,NaN,NaN,NaN
1726,Mario Tennis,gb,2000,sports,0.50,0.18,0.44,0.06,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,Assassin's Creed IV: Black Flag,xone,2013,action,1.48,0.55,0.00,0.21,NaN,7.4,M,средняя оценка,NaN
16175,Storm Lover 2nd,psp,2013,misc,0.00,0.00,0.02,0.00,NaN,NaN,NaN,NaN,NaN
727,Madden NFL 25,x360,2013,sports,1.98,0.06,0.00,0.19,80.0,5.6,E,средняя оценка,высокая оценка
16162,White Album 2: Shiawase no Mukougawa,psv,2013,adventure,0.00,0.00,0.02,0.00,NaN,NaN,NaN,NaN,NaN


- После категоризации данных проверим результат: сгруппируем данные по выделенным категориям и посчитаем количество игр в каждой категории.

In [34]:
grouped_user_df = df_actual.groupby('user_score_category')['name'].count()
grouped_critic_df = df_actual.groupby( 'critic_score_category')['name'].count()
print(f'Оценка пользователей {grouped_user_df}')
print(f'Оценка критиков {grouped_critic_df}')

Оценка пользователей user_score_category
низкая оценка      116
средняя оценка    4081
высокая оценка    2286
Name: name, dtype: int64
Оценка критиков critic_score_category
низкая оценка       55
средняя оценка    5422
высокая оценка    1692
Name: name, dtype: int64


- Выделим топ-7 платформ по количеству игр, выпущенных за весь актуальный период.

In [35]:
top7_platform = df_actual.groupby('platform')['name'].count().sort_values(ascending=False).head(7)
top7_platform

platform
ps2     2127
ds      2120
wii     1275
psp     1180
x360    1121
ps3     1087
gba      811
Name: name, dtype: int64

---

## Итоговый вывод

# В ходе проекта были нормализованы, отфильтрованы и категоризированы исходя из цели входные данные.
***Получены следующая информация по развитию игровой индустрии в начале 21 века (2000-2013 гг):***
- Количество игр по оценке пользователей: с низкой -- 116, средней -- 4081, высокой -- 2286).
- Количество игр по оценке критиков: с низкой -- 55, средней -- 5422, высокой -- 1692).
- Топ 7 платформ по количеству выпущенных игр: ps2 (2127), ds (2120), wii (1275), psp (1180), x360 (1121), ps3 (1087), gba (811).